# Lectura de datos

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
import numpy as np
import random
import time
import matplotlib.pyplot as plt

# Ruta de la carpeta de datos y resultados
data_folder = '/content/drive/MyDrive/MÁSTER/IC/P2/data/'
results_folder = '/content/drive/MyDrive/MÁSTER/IC/P2/resultados/'

# Establecer la semilla
seed = 2024
random.seed(seed)
np.random.seed(seed)

# Función para cargar los datos desde los archivos
def load_data(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Eliminar líneas vacías y espacios en blanco
    lines = [line.strip() for line in lines if line.strip()]

    # Extraer dimensiones
    n = int(lines[0])

    # Inicializar listas para flujo y distancias
    all_numbers = []

    # Leer todos los números
    for line in lines[1:]:
        all_numbers.extend(list(map(int, line.split())))

    # Dividir los números en las matrices de flujo y distancia
    total_numbers = len(all_numbers)
    if total_numbers != 2 * n * n:
        raise ValueError(f"El número de elementos ({total_numbers}) no coincide con el esperado para dos matrices {n}x{n} ({2*n*n}).")

    flow = np.array(all_numbers[:n*n]).reshape(n, n)
    distances = np.array(all_numbers[n*n:]).reshape(n, n)

    return flow, distances


# Algoritmo genético básico

In [5]:
def fitness_pop(population, flow, distances):
    return np.sum(flow[np.newaxis, :, :] * distances[population[:,:,np.newaxis], population[:, np.newaxis, :]], axis=(1, 2))

def tournament_selection(population, fitness_vals, tournament_size=5):
    selected = []
    for _ in range(len(population)):
        tournament = random.sample(range(len(population)), tournament_size)
        tournament_fitness = [fitness_vals[i] for i in tournament]
        winner = tournament[tournament_fitness.index(min(tournament_fitness))]
        selected.append(population[winner])
    return np.array(selected)

def ox_crossover(parent1, parent2):
    size = len(parent1)
    start, end = sorted(np.random.choice(range(size), 2, replace=False))

    child = -np.ones(size, dtype=int)
    child[start:end+1] = parent1[start:end+1]

    idx = 0
    for i in range(size):
        if child[i] == -1:
            while parent2[idx] in child:
                idx += 1
            child[i] = parent2[idx]
    return child

def swap_mutation(individual, mutation_prob=0.05):
    if random.random() < mutation_prob:
        i, j = np.random.choice(range(len(individual)), 2, replace=False)
        individual[i], individual[j] = individual[j], individual[i]
    return individual

def genetic_algorithm(flow, distances, population_size=1000, generations=500, crossover_prob=0.9, mutation_prob=0.05, tournament_size=5):
    num_cities = flow.shape[0]
    population = np.array([np.random.permutation(np.arange(num_cities)) for _ in range(population_size)])

    # Evaluación inicial de la población
    fitness_vals = fitness_pop(population, flow, distances)

    # Variables para control de generaciones sin mejora
    best_fitness = float('inf')
    generations_without_improvement = 0

    # Ciclo de generaciones
    for generation in range(generations):
        # Selección por torneo
        selected_population = tournament_selection(population, fitness_vals, tournament_size)

        # Cruzamiento (crossover)
        offspring = []
        for i in range(0, population_size, 2):
            if random.random() < crossover_prob:
                child1 = ox_crossover(selected_population[i], selected_population[i + 1])
                child2 = ox_crossover(selected_population[i + 1], selected_population[i])
            else:
                child1, child2 = selected_population[i], selected_population[i + 1]
            offspring.append(child1)
            offspring.append(child2)

        # Mutación
        population = np.array([swap_mutation(individual, mutation_prob) for individual in offspring])

        # Evaluación de la nueva población
        fitness_vals = fitness_pop(population, flow, distances)

        # Mejor individuo de esta generación
        current_best_fitness = np.min(fitness_vals)
        if current_best_fitness < best_fitness:
            best_fitness = current_best_fitness
            generations_without_improvement = 0
        else:
            generations_without_improvement += 1

        print(f"Generación {generation + 1}: Mejor coste = {current_best_fitness}")

        # Detener si no hay mejora en 15 generaciones consecutivas
        if generations_without_improvement >= 15:
            print(f"Terminando en la generación {generation + 1} por falta de mejora.")
            break

    # Resultados finales: Mejor solución y su coste
    best_idx = np.argmin(fitness_vals)
    return population[best_idx], fitness_vals[best_idx]

# Algoritmo Baldwiniano

In [6]:
def baldwinian_genetic_algorithm(flow, distances, population_size=1000, generations=500, crossover_prob=0.9, mutation_prob=0.05, tournament_size=5):
    def hill_climbing_optimized(individual):
        current = individual.copy()
        current_fitness = fitness_pop(np.array([current]), flow, distances)[0]
        for _ in range(100):
            i, j = np.random.choice(range(len(current)), 2, replace=False)
            neighbor = current.copy()
            neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
            delta_fitness = calculate_fitness_delta(current, i, j)
            if delta_fitness < 0:
                current = neighbor
                current_fitness += delta_fitness
        return current

    def calculate_fitness_delta(individual, i, j):
        delta = 0
        for k in range(len(individual)):
            if k != i and k != j:
                delta += (flow[i, k] - flow[j, k]) * (distances[individual[j], individual[k]] - distances[individual[i], individual[k]])
                delta += (flow[k, i] - flow[k, j]) * (distances[individual[k], individual[j]] - distances[individual[k], individual[i]])
        return delta

    population = np.array([np.random.permutation(np.arange(flow.shape[0])) for _ in range(population_size)])
    best_fitness = float('inf')
    generations_without_improvement = 0

    for generation in range(generations):
        fitness_vals = fitness_pop(population, flow, distances)

        # Mejora con Hill Climbing Baldwiniano
        top_individuals = population[np.argsort(fitness_vals)[:int(0.1 * population_size)]]
        improved_population = [hill_climbing_optimized(ind) for ind in top_individuals]

        # Reemplazar parte de la población con los mejores mejorados
        population[:len(improved_population)] = improved_population

        current_best_fitness = np.min(fitness_vals)

        if current_best_fitness < best_fitness:
            best_fitness = current_best_fitness
            generations_without_improvement = 0
        else:
            generations_without_improvement += 1

        print(f"Generación {generation + 1}: Mejor coste = {current_best_fitness}")

        if generations_without_improvement >= 15:
            print(f"Terminando en la generación {generation + 1} por falta de mejora.")
            break

    best_idx = np.argmin(fitness_vals)
    return population[best_idx], fitness_vals[best_idx]

# Algoritmo Lamarckiano

In [7]:
def lamarckian_genetic_algorithm(flow, distances, population_size=500, generations=200, crossover_prob=0.9, mutation_prob=0.05):
    def hill_climbing_optimized(individual):
        current = individual.copy()
        current_fitness = fitness_pop(np.array([current]), flow, distances)[0]
        for _ in range(100):
            i, j = np.random.choice(range(len(current)), 2, replace=False)
            neighbor = current.copy()
            neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
            delta_fitness = calculate_fitness_delta(current, i, j)
            if delta_fitness < 0:
                current = neighbor
                current_fitness += delta_fitness
        return current

    def calculate_fitness_delta(individual, i, j):
        delta = 0
        for k in range(len(individual)):
            if k != i and k != j:
                delta += (flow[i, k] - flow[j, k]) * (distances[individual[j], individual[k]] - distances[individual[i], individual[k]])
                delta += (flow[k, i] - flow[k, j]) * (distances[individual[k], individual[j]] - distances[individual[k], individual[i]])
        return delta

    population = np.array([np.random.permutation(np.arange(flow.shape[0])) for _ in range(population_size)])
    best_fitness = float('inf')
    generations_without_improvement = 0

    for generation in range(generations):
        # Aplicar Hill Climbing Lamarckiano a toda la población
        improved_population = [hill_climbing_optimized(ind) for ind in population]

        # Actualizar población con individuos optimizados
        population[:] = improved_population

        fitness_vals = fitness_pop(population, flow, distances)

        current_best_fitness = np.min(fitness_vals)

        if current_best_fitness < best_fitness:
            best_fitness = current_best_fitness
            generations_without_improvement = 0
        else:
            generations_without_improvement += 1

        print(f"Generación {generation + 1}: Mejor coste = {current_best_fitness}")

        if generations_without_improvement >= 15:
            print(f"Terminando en la generación {generation + 1} por falta de mejora.")
            break

    best_idx = np.argmin(fitness_vals)
    return population[best_idx], fitness_vals[best_idx]


# Ejecución del algoritmos (de golpe)

In [ ]:
# Función principal para ejecutar los experimentos
def run_experiments(folder_path):
    results = {}
    files = [f for f in os.listdir(folder_path) if f.endswith('.dat')]
    folder_path = os.path.abspath(folder_path)

    for file in files:
        print(f"Procesando archivo: {file}")
        file_path = os.path.join(folder_path, file)
        flow, distances = load_data(file_path)

        # Inicializar almacenamiento de resultados por fichero
        results[file] = {
            "standard": {"fitness": [], "time": 0},
            "baldwinian": {"fitness": [], "time": 0},
            "lamarckian": {"fitness": [], "time": 0}
        }

        # Ejecutar algoritmo estándar
        start_time = time.time()
        best_standard, fitness_standard = genetic_algorithm(flow, distances)
        results[file]["standard"]["time"] = time.time() - start_time
        results[file]["standard"]["fitness"] = fitness_standard

        # Ejecutar algoritmo baldwiniano
        start_time = time.time()
        best_baldwinian, fitness_baldwinian = baldwinian_genetic_algorithm(flow, distances)
        results[file]["baldwinian"]["time"] = time.time() - start_time
        results[file]["baldwinian"]["fitness"] = fitness_baldwinian

        # Ejecutar algoritmo lamarckiano
        start_time = time.time()
        best_lamarckian, fitness_lamarckian = lamarckian_genetic_algorithm(flow, distances)
        results[file]["lamarckian"]["time"] = time.time() - start_time
        results[file]["lamarckian"]["fitness"] = fitness_lamarckian

    # Comparar y visualizar resultados
    visualize_results(results)

def visualize_results(results):
    for file, data in results.items():
        print(f"Generando gráficos para: {file}")

        # Graficar evolución del fitness
        plt.figure(figsize=(12, 6))
        plt.plot(data["standard"]["fitness"], label="Estándar")
        plt.plot(data["baldwinian"]["fitness"], label="Baldwiniana")
        plt.plot(data["lamarckian"]["fitness"], label="Lamarckiana")
        plt.xlabel("Generación")
        plt.ylabel("Fitness (Mejor Solución)")
        plt.title(f"Evolución del Fitness - {file}")
        plt.legend()
        plt.grid()
        plt.savefig(os.path.join(results_folder, f"fitness_evolution_{file}.png"))
        plt.show()

        # Comparar tiempos de ejecución
        times = [
            data["standard"]["time"],
            data["baldwinian"]["time"],
            data["lamarckian"]["time"]
        ]
        algorithms = ["Estándar", "Baldwiniana", "Lamarckiana"]

        plt.figure(figsize=(8, 5))
        plt.bar(algorithms, times, color=["blue", "orange", "green"])
        plt.ylabel("Tiempo de Ejecución (s)")
        plt.title(f"Tiempos de Ejecución - {file}")
        plt.grid(axis="y")
        plt.savefig(os.path.join(results_folder, f"execution_times_{file}.png"))
        plt.show()

# Ejecutar experimentos
run_experiments(data_folder)

Procesando archivo: bur26a.dat
Generación 1: Mejor coste = 5600885
Generación 2: Mejor coste = 5579223
Generación 3: Mejor coste = 5579223


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-15f566a6b82a>", line 74, in <cell line: 0>
    run_experiments(data_folder)
  File "<ipython-input-6-15f566a6b82a>", line 21, in run_experiments
    best_standard, fitness_standard = genetic_algorithm(flow, distances)
                                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-3-aaee0981d08f>", line 54, in genetic_algorithm
    child1 = ox_crossover(selected_population[i], selected_population[i + 1])
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-3-aaee0981d08f>", line None, in ox_crossover
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiv

# Ejecución de algoritmos (dividida)

In [11]:
import pickle
import os

def run_experiments_part(folder_path, start_index, end_index, results_file):
    results = {}
    files = [f for f in os.listdir(folder_path) if f.endswith('.dat')]
    folder_path = os.path.abspath(folder_path)

    for file in files[start_index:end_index]:
        print(f"Procesando archivo: {file}")
        file_path = os.path.join(folder_path, file)
        flow, distances = load_data(file_path)

        results[file] = {
            "standard": {"fitness": [], "time": 0},
            "baldwinian": {"fitness": [], "time": 0},
            "lamarckian": {"fitness": [], "time": 0}
        }

        for algorithm in ["standard", "baldwinian", "lamarckian"]:
            start_time = time.time()
            if algorithm == "standard":
                best, fitness = genetic_algorithm(flow, distances)
            elif algorithm == "baldwinian":
                best, fitness = baldwinian_genetic_algorithm(flow, distances)
            else:
                best, fitness = lamarckian_genetic_algorithm(flow, distances)

            results[file][algorithm]["time"] = time.time() - start_time
            results[file][algorithm]["fitness"] = fitness

    # Guardar resultados en el archivo
    with open(results_file, 'wb') as f:
        pickle.dump(results, f)

def load_results(results_file):
    with open(results_file, 'rb') as f:
        return pickle.load(f)

In [12]:
# Define files before using it
files = [f for f in os.listdir(data_folder) if f.endswith('.dat')]

# Ejecutar la primera parte
run_experiments_part(data_folder, 0, len(files)//2, 'results_part1.pkl')

Procesando archivo: bur26a.dat
Generación 1: Mejor coste = 5600885
Generación 2: Mejor coste = 5579223
Generación 3: Mejor coste = 5579223
Generación 4: Mejor coste = 5578942
Generación 5: Mejor coste = 5570937
Generación 6: Mejor coste = 5530176
Generación 7: Mejor coste = 5550886
Generación 8: Mejor coste = 5548569
Generación 9: Mejor coste = 5520836
Generación 10: Mejor coste = 5500683
Generación 11: Mejor coste = 5522543
Generación 12: Mejor coste = 5498450
Generación 13: Mejor coste = 5498450
Generación 14: Mejor coste = 5483913
Generación 15: Mejor coste = 5493484
Generación 16: Mejor coste = 5488926
Generación 17: Mejor coste = 5487993
Generación 18: Mejor coste = 5481470
Generación 19: Mejor coste = 5481470
Generación 20: Mejor coste = 5476795
Generación 21: Mejor coste = 5471024
Generación 22: Mejor coste = 5468365
Generación 23: Mejor coste = 5468365
Generación 24: Mejor coste = 5468743
Generación 25: Mejor coste = 5467296
Generación 26: Mejor coste = 5467211
Generación 27: M

In [ ]:
results_part1 = load_results('results_part1.pkl')
run_experiments_part(data_folder, len(files)//2, len(files), 'results_part2.pkl')

# Combinar resultados
results_part2 = load_results('results_part2.pkl')
results = {**results_part1, **results_part2}

# Visualizar todos los resultados juntos
visualize_results(results)

Streaming output truncated to the last 5000 lines.
Generación 2: Mejor coste = 2022136
Generación 3: Mejor coste = 2018684
Generación 4: Mejor coste = 2018684
Generación 5: Mejor coste = 1969432
Generación 6: Mejor coste = 2012858
Generación 7: Mejor coste = 1988130
Generación 8: Mejor coste = 1988130
Generación 9: Mejor coste = 1976224
Generación 10: Mejor coste = 1988858
Generación 11: Mejor coste = 1941626
Generación 12: Mejor coste = 1923470
Generación 13: Mejor coste = 1961978
Generación 14: Mejor coste = 1971424
Generación 15: Mejor coste = 1966574
Generación 16: Mejor coste = 1880212
Generación 17: Mejor coste = 1880212
Generación 18: Mejor coste = 1880212
Generación 19: Mejor coste = 1880212
Generación 20: Mejor coste = 1880212
Generación 21: Mejor coste = 1889740
Generación 22: Mejor coste = 1889740
Generación 23: Mejor coste = 1889740
Generación 24: Mejor coste = 1889740
Generación 25: Mejor coste = 1889740
Generación 26: Mejor coste = 1889740
Generación 27: Mejor coste = 188

NameError: name 'visualize_results' is not defined